<a href="https://colab.research.google.com/github/kazuki-komori/coursera/blob/main/regression/CSR_regression_w3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 第3回 勉強会

## 重回帰のための勾配降下法の実装
Implementing gradient descent for multiple regression

### TODO
- 切片を考慮し、SFrame（またはそれ以外）に1の定数列を追加する

  > Add a constant column of 1's to a SFrame (or otherwise) to account for the intercept

- SFrameをnumpyの配列に変換する

  > Convert an SFrame into a numpy array

- numpy を使って predict_output() 関数を書く

  > Write a predict_output() function using numpy

- 単一の特徴量に関する回帰重みの導関数を計算するnumpy関数を記述する

  > Write a numpy function to compute the derivative of the regression weights with respect to a single feature

- 初期重みベクトル、ステップサイズ、許容誤差を与えて、回帰重みを計算する勾配降下関数を書く

  > Write gradient descent function to compute the regression weights given an initial weight vector, step size and tolerance.

- 複数の特徴量の回帰重みを推定するために勾配降下関数を使用する
  
  > Use the gradient descent function to estimate regression weights for multiple features


## setup

In [ ]:
!unzip -q /content/drive/MyDrive/coursera/home_data.zip

In [ ]:
!pip install turicreate

     |████████████████████████████████| 92.0 MB 617 kB/s 
     |████████████████████████████████| 322 kB 44.4 MB/s 
     |████████████████████████████████| 3.5 MB 45.1 MB/s 
     |████████████████████████████████| 3.6 MB 52.6 MB/s 
     |████████████████████████████████| 86.4 MB 72 kB/s 
     |████████████████████████████████| 18.3 MB 1.3 MB/s 
     |████████████████████████████████| 2.9 MB 52.7 MB/s 
     |████████████████████████████████| 20.1 MB 51.4 MB/s 
     |████████████████████████████████| 3.8 MB 55.1 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 449 kB 56.4 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=e30790ed21b612882d747508c5088e7e8b90cee394b97f45a2acca0f2c5ccb52
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320859 sha256=3ae9a20cc

In [ ]:
import turicreate
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

## (1) データの読み込み

In [ ]:
df_house = turicreate.SFrame('/content/home_data.sframe')
df_house.head(5)

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650.0,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242.0,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000.0,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000.0,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080.0,1.0,0
view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat
0,3,7.0,1180.0,0.0,1955.0,0.0,98178,47.51123398
0,3,7.0,2170.0,400.0,1951.0,1991.0,98125,47.72102274
0,3,6.0,770.0,0.0,1933.0,0.0,98028,47.73792661
0,5,7.0,1050.0,910.0,1965.0,0.0,98136,47.52082


## (2) numpy の import

In [ ]:
import numpy as np

## (3) SFrame から numpry に変換する

In [ ]:
def get_numpy_data(df, features, output):
  df['constant'] = 1
  features_matrix = df[[*features, "constant"]].to_numpy()
  output_array = df[output].to_numpy()
  return features_matrix, output_array

## (4) 1次元重みを付与し, 予測値を算出

In [ ]:
def predict_outcome(feature_matrix, weights):
  predictions = np.dot(feature_matrix, weights)
  return predictions

hoge fuga piyo


## (5) 誤差配列を受け取ってを1位の値を算出してみる

In [ ]:
def feature_derivative(errors, feature):
    [your code here]
    return(derivative)